# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Lab 8.01 | Topic Modeling & Network Analysis

In [ ]:
!pip install bokeh

**Problem 1:** Define supervised learning and unsupervised learning.

**Answer:** no target with unsupervised learning

**Problem 2:** Give an example of a supervised learning problem.

**Answer:** try to figure out score to a knicks game

**Problem 3:** Give an example of an unsupervised learning problem.

**Answer:** topics within a google search throughout the day

**Problem 4:** Imagine you're in a non-technical interview with a non-technical recruiter. How would you define supervised learning and unsupervised learning?

**Answer:** Say we are classifying triangles, squares, and diamonds. Unsupervised learning is grouping these shapes without ever seeing them before. Supervised learning is where we know what a triangle, square and diamond is and then we can classify the shapes, based on number of verticies, and angle size.

In [34]:
import pandas as pd
import numpy as np
import sklearn.datasets
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from gensim import models, corpora

In [2]:
newsgroups_train = sklearn.datasets.fetch_20newsgroups(subset='train')

In [3]:
train = pd.DataFrame(newsgroups_train.data, columns=["text"])

In [4]:
train.loc[0,"text"]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [13]:
text_process(train.loc[0,"text"])

['lerxst',
 'wam',
 'umd',
 'edu',
 'thing',
 'subject',
 'car',
 'nntp',
 'post',
 'host',
 'rac3',
 'wam',
 'umd',
 'edu',
 'organ',
 'univers',
 'maryland',
 'colleg',
 'park',
 'line',
 '15',
 'wonder',
 'anyon',
 'could',
 'enlighten',
 'car',
 'saw',
 'day',
 '2',
 'door',
 'sport',
 'car',
 'look',
 'late',
 '60',
 'earli',
 '70',
 'call',
 'bricklin',
 'door',
 'realli',
 'small',
 'addit',
 'front',
 'bumper',
 'separ',
 'rest',
 'bodi',
 'know',
 'anyon',
 'tellm',
 'model',
 'name',
 'engin',
 'spec',
 'year',
 'product',
 'car',
 'made',
 'histori',
 'whatev',
 'info',
 'funki',
 'look',
 'car',
 'pleas',
 'e',
 'mail',
 'thank',
 'il',
 'brought',
 'neighborhood',
 'lerxst']

**Problem 5:** Build an LDA model on the training data with 20 topics. Summarize any findings. Be sure to preprocess your data accordingly!
- Note that pre-processing your data will likely take ten or so minutes; if you use a loop to pre-process your data, you should consider adding some `print` statement that will let you know every 1,000 entries that you are indeed making progress.
- Fitting the LDA model won't take as long, but will still take longer given the larger size of the data.

In [ ]:
np.random.seed(42)

In [6]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- one small tweak

In [53]:
texts = []
for i in range(500):
    texts.append(text_process(train.iloc[i,:].values[0]))

In [54]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [55]:
ldamodel = models.ldamodel.LdaModel(corpus, 
                                    id2word = dictionary, 
                                    num_topics = 20, 
                                    passes = 5, 
                                    minimum_probability=0)

In [52]:
ldamodel.get_topics()

array([[ 0.00342079,  0.00339651,  0.0034052 , ...,  0.00016941,
         0.00016941,  0.00016941],
       [ 0.00130739,  0.00165799,  0.00125329, ...,  0.00125329,
         0.00125329,  0.00125329],
       [ 0.00135328,  0.00135685,  0.00135275, ...,  0.00135275,
         0.00135275,  0.00135275],
       ..., 
       [ 0.00290747,  0.02225376,  0.00014413, ...,  0.00014413,
         0.00014413,  0.00014413],
       [ 0.00160638,  0.00218157,  0.00175404, ...,  0.00125447,
         0.00125447,  0.00125447],
       [ 0.001653  ,  0.00315747,  0.00152491, ...,  0.00123006,
         0.00126849,  0.00122341]], dtype=float32)

**Problem 6:** Let's walk through using a cool visualization library, called [Bokeh](https://bokeh.pydata.org). 

**6.1:** Run the following cell. This will convert our LDA results into an array of size `number_of_documents` $\times$ `number_of_topics`. Remember that each document is a combination of topics; this code shows us the distribution of topics for a given document.

In [56]:
doc_x_topic = np.array([[y for (x,y) in ldamodel[corpus[i]]] for i in range(len(corpus))])

**6.2:** Run the following cell. 

Remember that very topic is defined by all of the words in our corpus. For example, the topic magic, might be defined by the words {spell, wand, wizard}. The topic magic, then, is defined in three dimensions.

If we have a corpus of more than three words (which is going to basically be 100% of the time), then it will be very difficult to visualize our topics.
- In using `pyLDAvis.gensim.prepare()`, the function "projected" our data from high dimensions into only two dimensions using something called **principal component analysis**. We'll learn about PCA later this week!
- There's another commonly used method of projecting data into lower dimensions, called [**t-Stochastic Neighbor Embedding**](https://lvdmaaten.github.io/tsne/). We won't go into the details here, but you're welcome to learn more about it if you'd like. We're just using it so that it makes visualizations easier.

Run the following cell. It'll implement t-SNE for us!

In [57]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42, perplexity=30)        # Instantiates Model
tsne_embedding = tsne.fit_transform(doc_x_topic)   # Fits Model/Transforms the Data
tsne_embedding = pd.DataFrame(tsne_embedding, columns=['x','y']) # Creates Pandas Dataframe of TSNE results.
tsne_embedding['hue'] = doc_x_topic.argmax(axis=1) # Creates "hue" based on likeliest topic.

In the absence of any metrics telling us whether we have a "good" or "bad" model, this will help us to better visualize the performance of our model!

**6.3:** Run the following two cells.

In [58]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider
from bokeh.layouts import column
from bokeh.palettes import all_palettes
output_notebook()

Loading BokehJS ...

In [59]:
source = ColumnDataSource(
        data=dict(
            x = tsne_embedding.x,
            y = tsne_embedding.y,
            colors = [all_palettes['Category20'][20][i] for i in tsne_embedding.hue],
            topic = [i for i in tsne_embedding.hue],
            alpha = [0.7] * tsne_embedding.shape[0],
            size = [5] * tsne_embedding.shape[0]
        )
    )

hover_tsne = HoverTool(names=["train"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Topic:</span>
            <span style="font-size: 12px">@topic</span>
        </div>
    </div>
    """)
tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
plot_tsne = figure(plot_width=700, plot_height=700, tools=tools_tsne, title='Newsgroups')
plot_tsne.circle('x', 'y', size='size', fill_color='colors', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, name="train")

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    x = data['x']
    y = data['y']
    colors = data['colors']
    alpha = data['alpha']
    size = data['size']
    for (i = 0; i < x.length; i++) {
        alpha[i] = 0.7
        size[i] = 5
    }
    source.trigger('change');
""")

In [60]:
show(plot_tsne)

Bokeh is a great library for interactive visualizations! It offers us a lot of flexibility to generate visualizations for ourselves or for stakeholders. While knowing Javascript is helpful, the [Bokeh gallery](https://bokeh.pydata.org/en/latest/docs/gallery.html) and the [Bokeh reference guide](https://bokeh.pydata.org/en/latest/docs/reference.html) are very helpful to those of us who don't know JavaScript.

It also works really well with other packages, including NetworkX!

**Problem 7:** Go through the next cell and comment each line. What does each line appear to be doing?

In [62]:
import networkx as nx

from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

G=nx.karate_club_graph()

plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

output_file("networkx_graph.html")
show(plot)

**Problem 8:** Run the next cell. What are the benefits to having a visualization like this?

In [63]:
import networkx as nx

from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4

G=nx.karate_club_graph()

plot = Plot(plot_width=400, plot_height=400,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Graph Interaction Demonstration"

plot.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)

**Answer:**

**Problem 9:** Referring to the previous cell, what might be a case when the previous visualization is not helpful?

**Answer:**

**Problem 10:** Create at least two more interactive graphs in Bokeh. They can be networks, topic models, or something else... perhaps related to your upcoming project??

**Answer:**